## Numpy 进阶

在本附录中，我将更深入地研究用于数组计算的NumPy库。这将包括有关ndarray类型的更多内部细节以及更高级的数组操作和算法。

本附录包含其他主题，不一定需要线性阅读。

## A.1 ndarray对象内部

NumPy ndarray提供了一种手段，可以将同构数据块（连续或跨步）解释为多维数组对象。数据类型（或dtype）确定如何将数据解释为浮点型，整数型，布尔型或我们一直关注的任何其他类型。

使ndarray灵活的部分原因在于，每个数组对象都是数据块上的跨步视图。例如，您可能想知道数组视图arr [:: 2，::-1]如何不复制任何数据。 原因是ndarray不仅仅是一块内存和一个dtype；它还具有“跨步”信息，使阵列能够以不同的步长在内存中移动。更准确地说，ndarray内部包含以下内容：

- 指向数据的指针，即RAM或内存映射文件中的数据块
- 数据类型或dtype，描述数组中固定大小的值单元格
- 表示数组形状的元组
- 跨度的元组，整数表示要沿维度前进一个元素的要“步进”的字节数

有关ndarray内部模型的简单模型，请参见图A-1。

![app1](images/app.1.png)

例如，一个10×5的数组将具有形状（10，5）：

In [1]:
import numpy as np

np.ones((10, 5)).shape

(10, 5)

一个典型的（C阶）3x4×5数组float64（8字节）值具有步幅（160、40、8）（了解这些步幅可能很有用，因为通常，特定轴上的步幅较大 ，沿该轴执行计算的成本更高）：

In [2]:
np.ones((3, 4, 5), dtype=np.float64).strides

(160, 40, 8)

虽然很少有典型的NumPy用户对数组的跨步感兴趣，但它们是构造“零复制”数组视图的关键要素。跨度甚至可以是负数，从而使数组可以在内存中“向后移动”（例如，在obj [::-1]或obj [:, ::-1]这样的切片中就是这种情况）。

### NumPy dtype层次结构

您可能偶尔会有需要检查数组是否包含整数，浮点数，字符串或Python对象的代码。因为有多种类型的浮点数（float16到float128），所以检查dtype是否在类型列表中会非常冗长。幸运的是，dtypes具有超类，例如np.integer和np.floating，它们可以与np.issubdtype函数结合使用：

In [3]:
ints = np.ones(10, dtype=np.uint64)

In [4]:
floats = np.ones(10, dtype=np.float32)

In [5]:
np.issubdtype(ints.dtype, np.integer)

True

In [6]:
np.issubdtype(floats.dtype, np.floating)

True

您可以通过调用类型的mro方法查看特定dtype的所有父类：

In [7]:
np.float64.mro()

[numpy.float64,
 numpy.floating,
 numpy.inexact,
 numpy.number,
 numpy.generic,
 float,
 object]

因此，我们还有：

In [8]:
np.issubdtype(ints.dtype, np.number)

True

大多数NumPy用户永远都不必知道这一点，但它有时会派上用场。有关dtype层次结构和父子类关系的图表，请参见图A-2。

![app2](images/app.2.png)

## A.2 矩阵操作进阶

除了花式的索引，切片和布尔子集以外，还有许多方法可以使用数组。尽管数据分析应用程序的许多繁重工作都是由熊猫中的高级功能来处理的，但您有时可能需要编写一种在现有库中找不到的数据算法。

### 调整数据形状

在许多情况下，您可以将数组从一种形状转换为另一种形状，而无需复制任何数据。为此，将表示新形状的元组传递给reshape数组实例方法。例如，假设我们有一个一维值数组，希望将其重新排列成矩阵（结果如图A-3所示）：

![app4](images/app.3.png)

In [9]:
arr = np.arange(8)

arr

array([0, 1, 2, 3, 4, 5, 6, 7])

In [10]:
arr.reshape((4, 2))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

多维数组也可以调整：

In [11]:
arr.reshape((4, 2)).reshape((2, 4))

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

传递的形状尺寸之一可以是–1，在这种情况下，将从数据推断用于该尺寸的值：

In [12]:
arr = np.arange(15)

arr.reshape((5, -1))

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

由于数组的shape属性是一个元组，因此也可以将其传递给整形：

In [13]:
other_arr = np.ones((3, 5))
other_arr.shape

(3, 5)

In [14]:
arr.reshape(other_arr.shape)

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

从一维到高维的相反整形操作通常称为展平或拉平：

In [15]:
arr = np.arange(15).reshape((5, 3))

arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [16]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

如果结果中的值在原始数组中是连续的，那么ravel不会生成基础值的副本。flatten方法的行为类似于ravel，但它始终返回数据的副本：

In [17]:
arr.flatten()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

数据可以按不同顺序重塑或整理。对于新的NumPy用户而言，这是一个微妙的主题，因此是下一个子主题。

### C与Fortran顺序

NumPy使您可以控制和灵活地控制数据在内存中的布局。默认情况下，按行主要顺序创建NumPy数组。从空间上讲，这意味着如果您有一个二维数据数组，则该数组每一行中的项目都存储在相邻的内存位置中。行主要顺序的替代方法是列主要顺序，这意味着每列数据中的值都存储在相邻的存储位置中。

由于历史原因，行和列的主要顺序也分别称为C和Fortran顺序。在FORTRAN 77语言中，矩阵都是主列。

诸如reshape和ravel之类的函数接受一个order参数，该参数指示使用数组中数据的顺序。在大多数情况下，通常将其设置为“ C”或“ F”（也有一些不常用的选项“ A”和“ K”；请参见NumPy文档，并参考图A-3以获得这些选项的说明） ）：

In [18]:
arr = np.arange(12).reshape((3, 4))

arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [19]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [20]:
arr.ravel('F')

array([ 0,  4,  8,  1,  5,  9,  2,  6, 10,  3,  7, 11])

重塑具有两个以上维度的数组可能会有些弯腰（请参见图A-3）。C和Fortran顺序之间的主要区别在于维度的行走方式：

> C/行主要顺序
    首先移动较大的尺寸（例如，在轴0前进之前先在轴1上移动）。
> Fortran/列主要顺序
    最后移动较大的尺寸（例如，沿轴1前进之前的轴0）。
    
### 连接和拆分数组

numpy.concatenate接受数组的序列（元组，列表等），并沿输入轴按顺序将它们连接在一起：

In [21]:
arr1 = np.array([[1, 2, 3], [4, 5, 6]])
arr2 = np.array([[7, 8, 9], [10, 11, 12]])

In [22]:
np.concatenate([arr1, arr2], axis=0)

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [23]:
np.concatenate([arr1, arr2], axis=1)

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

有一些方便的功能，例如vstack和hstack，用于常见的串联。前面的操作可以表示为：

In [24]:
np.vstack((arr1, arr2))

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [25]:
np.hstack((arr1, arr2))

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

split, 另一方面，将一个数组沿一个轴切成多个数组：

In [26]:
arr = np.random.randn(5, 2)

In [27]:
arr

array([[-1.61901755, -1.9332683 ],
       [ 0.25073952, -1.03502051],
       [-0.33103284,  0.61901676],
       [ 0.44292818,  0.85685508],
       [-0.94375608, -0.63382524]])

In [28]:
first, second, third = np.split(arr, [1, 3])

In [29]:
first

array([[-1.61901755, -1.9332683 ]])

In [30]:
second

array([[ 0.25073952, -1.03502051],
       [-0.33103284,  0.61901676]])

In [31]:
third

array([[ 0.44292818,  0.85685508],
       [-0.94375608, -0.63382524]])

传递给np.split的[1，3]值指示将数组拆分为多个片段的索引。

有关所有相关级联和拆分功能的列表，请参见表A-1，其中某些功能仅出于非常通用的级联的便利而提供。

#### Stacking 帮助: r_ and c_

NumPy命名空间中有两个特殊对象r_和c_，它们使堆栈数组更简洁：

In [32]:
arr = np.arange(6)

arr1 = arr.reshape((3, 2))
arr2 = np.random.randn(3, 2)

np.r_[arr1, arr2]

array([[ 0.        ,  1.        ],
       [ 2.        ,  3.        ],
       [ 4.        ,  5.        ],
       [ 0.2147942 ,  1.97260327],
       [ 0.39373445, -0.52875549],
       [-0.17477746,  1.27464435]])

In [33]:
np.c_[np.r_[arr1, arr2], arr]

array([[ 0.        ,  1.        ,  0.        ],
       [ 2.        ,  3.        ,  1.        ],
       [ 4.        ,  5.        ,  2.        ],
       [ 0.2147942 ,  1.97260327,  3.        ],
       [ 0.39373445, -0.52875549,  4.        ],
       [-0.17477746,  1.27464435,  5.        ]])

这些还可以将切片转换为数组：

In [34]:
np.c_[1:6, -10:-5]

array([[  1, -10],
       [  2,  -9],
       [  3,  -8],
       [  4,  -7],
       [  5,  -6]])

有关使用c_和r_可以执行的操作的更多信息，请参见文档字符串。

### 重复元素：平铺并重复

用于重复或复制数组以生成更大数组的两个有用的工具是repeat和tile功能。repeat多次复制数组中的每个元素，从而产生更大的数组：

In [35]:
arr = np.arange(3)

arr

array([0, 1, 2])

In [36]:
arr.repeat(3)

array([0, 0, 0, 1, 1, 1, 2, 2, 2])

> 与其他数组编程框架（如MATLAB）相比，NumPy很少需要复制或重复数组。原因之一是广播通常可以更好地满足这一需求，这是下一部分的主题。

默认情况下，如果传递整数，则每个元素将重复该次数。如果传递整数数组，则每个元素可以重复不同的次数：

In [37]:
arr.repeat([2, 3, 4])

array([0, 0, 1, 1, 1, 2, 2, 2, 2])

多维数组可以使其元素沿特定轴重复。

In [38]:
arr = np.random.randn(2, 2)

arr

array([[ 0.44663835, -1.20995865],
       [ 0.66875485,  0.59349705]])

In [39]:
arr.repeat(2, axis=0)

array([[ 0.44663835, -1.20995865],
       [ 0.44663835, -1.20995865],
       [ 0.66875485,  0.59349705],
       [ 0.66875485,  0.59349705]])

In [40]:
arr.repeat(2, axis=1)

array([[ 0.44663835,  0.44663835, -1.20995865, -1.20995865],
       [ 0.66875485,  0.66875485,  0.59349705,  0.59349705]])

请注意，如果没有传递轴，则数组将首先被展平，这可能不是您想要的。类似地，当重复多维数组以给定切片重复不同次数时，可以传递整数数组：

In [41]:
arr.repeat([2, 3], axis=0)

array([[ 0.44663835, -1.20995865],
       [ 0.44663835, -1.20995865],
       [ 0.66875485,  0.59349705],
       [ 0.66875485,  0.59349705],
       [ 0.66875485,  0.59349705]])

In [42]:
arr.repeat([2, 3], axis=1)

array([[ 0.44663835,  0.44663835, -1.20995865, -1.20995865, -1.20995865],
       [ 0.66875485,  0.66875485,  0.59349705,  0.59349705,  0.59349705]])

另一方面，tile是沿轴堆叠数组副本的快捷方式。在视觉上，您可以认为它类似于“铺设瓷砖”：

In [43]:
arr

array([[ 0.44663835, -1.20995865],
       [ 0.66875485,  0.59349705]])

In [44]:
np.tile(arr, 2)

array([[ 0.44663835, -1.20995865,  0.44663835, -1.20995865],
       [ 0.66875485,  0.59349705,  0.66875485,  0.59349705]])

第二个参数是瓦片的数量；使用标量时，平铺会逐行进行，而不是逐列进行。tile的第二个参数可以是表示“平铺”布局的元组：

In [45]:
arr

array([[ 0.44663835, -1.20995865],
       [ 0.66875485,  0.59349705]])

In [46]:
np.tile(arr, (2, 1))

array([[ 0.44663835, -1.20995865],
       [ 0.66875485,  0.59349705],
       [ 0.44663835, -1.20995865],
       [ 0.66875485,  0.59349705]])

In [47]:
np.tile(arr, (3, 2))

array([[ 0.44663835, -1.20995865,  0.44663835, -1.20995865],
       [ 0.66875485,  0.59349705,  0.66875485,  0.59349705],
       [ 0.44663835, -1.20995865,  0.44663835, -1.20995865],
       [ 0.66875485,  0.59349705,  0.66875485,  0.59349705],
       [ 0.44663835, -1.20995865,  0.44663835, -1.20995865],
       [ 0.66875485,  0.59349705,  0.66875485,  0.59349705]])

### 花式索引等效项：take和put

您可能从第4章中回忆过，获取和设置数组子集的一种方法是使用整数数组进行华丽索引：

In [48]:
arr = np.arange(10) * 100

inds = [7, 1, 2, 6]
arr[inds]

array([700, 100, 200, 600])

在仅在单个轴上进行选择的特殊情况下，还有其他ndarray方法很有用：

In [49]:
arr.take(inds)

array([700, 100, 200, 600])

In [50]:
arr.put(inds, 42)

In [51]:
arr

array([  0,  42,  42, 300, 400, 500,  42,  42, 800, 900])

In [52]:
arr.put(inds, [40, 41, 42, 43])

In [53]:
arr

array([  0,  41,  42, 300, 400, 500,  43,  40, 800, 900])

要沿其他轴使用汇整，可以传递axis关键字：

In [54]:
inds = [2, 0, 2, 1]
arr = np.random.randn(2, 4)

In [55]:
arr

array([[ 0.47172788, -1.01467909, -0.62255594, -1.15780264],
       [ 0.39554734, -0.84808447,  0.32981324,  0.28452574]])

In [56]:
arr.take(inds, axis=1)

array([[-0.62255594,  0.47172788, -0.62255594, -1.01467909],
       [ 0.32981324,  0.39554734,  0.32981324, -0.84808447]])

put不接受轴参数，而是索引到数组的展平（一维，C阶）版本。因此，当您需要使用其他轴上的索引数组设置元素时，通常最容易使用花式索引。

## A.3 广播

广播描述了算术如何在不同形状的数组之间工作。它可能是一项强大的功能，但即使对于有经验的用户，也可能引起混乱。最简单的广播示例是将标量值与数组组合在一起时发生的：

In [57]:
arr = np.arange(5)

In [58]:
arr

array([0, 1, 2, 3, 4])

In [59]:
arr * 4

array([ 0,  4,  8, 12, 16])

在这里，我们说标量值4已在乘法运算中广播到所有其他元素。

例如，我们可以通过减去列均值来减少数组的每一列。在这种情况下，这非常简单：

In [60]:
arr = np.random.randn(4, 3)

arr.mean(0)

array([ 0.29631735, -0.64023109, -0.32615285])

In [61]:
demeaned = arr - arr.mean(0)

In [62]:
demeaned

array([[-0.3871364 , -0.93325951,  0.46995205],
       [ 0.17072873,  1.12699773, -0.12339537],
       [ 0.44921085, -1.05789749,  0.64636361],
       [-0.23280318,  0.86415928, -0.99292029]])

In [63]:
demeaned.mean(0)

array([-1.38777878e-17,  0.00000000e+00, -2.77555756e-17])

有关此操作的说明，请参见图A-4。将行取消为广播操作需要多加注意。幸运的是，只要遵循规则，就有可能在数组的任何维度上广播潜在的较低维度值（例如，从二维数组的每一列中减去行均值）。

![app4](images/app.4.png)

这使我们能够：

> 广播规则
> 如果对于每个尾随尺寸（即，从末端开始），轴长度匹配，或者如果两个长度中的任何一个为1，则两个阵列都可以进行广播。然后在丢失或长度为1的尺寸上执行广播。

即使是经验丰富的NumPy用户，当我考虑广播规则时，我也经常不得不停下来画一个图。考虑上一个示例，并假设我们希望从每行中减去平均值。由于arr.mean（0）的长度为3，因此它兼容跨轴0广播，因为arr中的尾随尺寸为3，因此匹配。根据规则，要在轴1上减去（即从每行中减去行均值），较小的数组必须具有形状（4，1）：

In [64]:
arr

array([[-0.09081904, -1.5734906 ,  0.1437992 ],
       [ 0.46704608,  0.48676664, -0.44954823],
       [ 0.7455282 , -1.69812858,  0.32021075],
       [ 0.06351417,  0.22392819, -1.31907314]])

In [65]:
row_means = arr.mean(1)

In [66]:
row_means.shape

(4,)

In [67]:
row_means.reshape((4, 1))

array([[-0.50683681],
       [ 0.16808816],
       [-0.21079654],
       [-0.34387693]])

In [68]:
demeaned = arr - row_means.reshape((4, 1))

In [69]:
demeaned.mean(1)

array([1.11022302e-16, 0.00000000e+00, 0.00000000e+00, 3.70074342e-17])

有关此操作的说明，请参见图A-5。

![app5](images/app.5.png)

参见图A-6，这是另一个说明，这一次将一个二维数组添加到一个跨轴0的三维数组中。

![app6](images/app.6.png)

### 通过其他轴广播

使用高维数组进行广播似乎更加令人费解，但这实际上是遵循规则的问题。如果不这样做，则会出现如下错误：

In [70]:
arr - arr.mean(1)

ValueError: operands could not be broadcast together with shapes (4,3) (4,) 

想要对跨轴（而不是轴0）的较低维度数组执行算术运算是很常见的。根据广播规则，较小数组中的“广播维度”必须为1。在此处显示的行贬低示例中，这意味着将行重塑为形状（4，1）而不是（4，）：

In [71]:
arr - arr.mean(1).reshape((4, 1))

array([[ 0.41601777, -1.06665378,  0.65063601],
       [ 0.29895792,  0.31867847, -0.61763639],
       [ 0.95632474, -1.48733204,  0.5310073 ],
       [ 0.4073911 ,  0.56780512, -0.97519621]])

在三维情况下，在三个维度中的任何一个上进行广播仅是将数据重塑为形状兼容的问题。图A-7很好地可视化了在三维阵列的每个轴上广播所需的形状。

![app7](images/app.7.png)

因此，一个常见的问题是需要专门为广播目的添加一个长度为1的新轴。使用重塑是一种选择，但是插入轴需要构造一个指示新形状的元组。这通常是一个乏味的练习。因此，NumPy数组为通过索引插入新轴提供了一种特殊的语法。 我们使用特殊的np.newaxis属性以及“ full”切片来插入新轴：

In [72]:
arr = np.zeros((4, 4))

In [73]:
arr_3d = arr[:, np.newaxis, :]

In [74]:
arr_3d.shape

(4, 1, 4)

In [75]:
arr_1d = np.random.normal(size=3)

In [76]:
arr_1d[:, np.newaxis]

array([[-0.6436621 ],
       [ 0.1879725 ],
       [-0.56895891]])

In [77]:
arr_1d[np.newaxis, :]

array([[-0.6436621 ,  0.1879725 , -0.56895891]])

因此，如果我们有一个三维数组并想降低轴2的强度，则需要编写：

In [78]:
arr = np.random.randn(3, 4, 5)

In [79]:
depth_means = arr.mean(2)

In [80]:
depth_means

array([[ 0.47335151, -0.13097157, -0.45077735,  0.26695798],
       [ 0.50734862,  0.25599043, -0.20621867, -0.13400995],
       [-0.17265517, -0.61267556,  0.06057995, -0.77499787]])

In [81]:
depth_means.shape

(3, 4)

In [82]:
demeaned = arr - depth_means[:, :, np.newaxis]

In [83]:
demeaned.mean(2)

array([[ 2.22044605e-17,  1.11022302e-17, -2.22044605e-17,
         0.00000000e+00],
       [ 8.88178420e-17, -4.44089210e-17,  1.66533454e-17,
         0.00000000e+00],
       [ 1.11022302e-17, -2.22044605e-17,  0.00000000e+00,
        -2.22044605e-17]])

您可能想知道是否有一种方法可以在不牺牲性能的情况下在轴上进行通用归位。有，但是需要一些索引体操：

In [84]:
def demean_axis(arr, axis=0):
    means = arr.mean(axis)
    
    # This generalize things like [:, :, np.newaxis] to N dimensions
    indexer = [slice(None)] * arr.ndim
    indexer[axis] = np.newaxis
    return arr - means[tuple(indexer)]

In [85]:
demean_axis(arr).mean(0)

array([[-3.70074342e-17,  2.22044605e-16,  1.85037171e-17,
        -9.25185854e-18,  4.62592927e-17],
       [-3.70074342e-17,  7.40148683e-17,  9.25185854e-18,
         0.00000000e+00, -1.85037171e-17],
       [ 3.70074342e-17,  0.00000000e+00,  1.11022302e-16,
         1.85037171e-17, -1.85037171e-17],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -7.40148683e-17,  3.70074342e-17]])

### 通过广播设置数组值

相同的控制算术运算的广播规则也适用于通过数组索引设置值。在简单的情况下，我们可以执行以下操作：

In [86]:
arr = np.zeros((4, 3))

In [87]:
arr[:] = 5

In [88]:
arr

array([[5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.]])

但是，如果我们想将一维值数组设置为数组的列，那么只要形状兼容，我们就可以这样做：

In [89]:
col = np.array([1.28, -0.42, 0.44, 1.6])

In [90]:
arr[:] = col[:, np.newaxis]

In [91]:
arr

array([[ 1.28,  1.28,  1.28],
       [-0.42, -0.42, -0.42],
       [ 0.44,  0.44,  0.44],
       [ 1.6 ,  1.6 ,  1.6 ]])

In [92]:
arr[:2] = [[-1.37], [0.509]]

In [93]:
arr

array([[-1.37 , -1.37 , -1.37 ],
       [ 0.509,  0.509,  0.509],
       [ 0.44 ,  0.44 ,  0.44 ],
       [ 1.6  ,  1.6  ,  1.6  ]])

## A.4 高级ufunc用法

虽然许多NumPy用户将仅使用通用函数提供的快速的按元素操作，但仍有许多其他功能可以偶尔帮助您编写更简洁的代码而无需循环。

### ufunc实例方法

NumPy的每个二进制ufunc都有执行某些特殊矢量化操作的特殊方法。表A-2中总结了这些内容，但我将给出一些具体示例来说明它们的工作方式。

reduce执行单个二进制数组，并通过执行一系列二进制运算来汇总其值（可选地沿一个轴）。例如，对数组中的元素求和的另一种方法是使用np.add.reduce：

In [94]:
arr = np.arange(10)

In [95]:
np.add.reduce(arr)

45

In [96]:
arr.sum()

45

起始值（0用于添加）取决于ufunc。如果通过了轴，则将沿该轴进行缩小。使您能够以简洁的方式回答某些类型的问题。举一个简单的例子，我们可以使用np.logical_and来检查数组每一行中的值是否已排序：

In [97]:
np.random.seed(12346) # for reproducibity

In [98]:
arr = np.random.randn(5, 5)

In [99]:
arr[::2].sort(1) # sort a few rows

In [100]:
arr[:, :-1] < arr[:, 1:]

array([[ True,  True,  True,  True],
       [False,  True, False, False],
       [ True,  True,  True,  True],
       [ True, False,  True,  True],
       [ True,  True,  True,  True]])

In [101]:
np.logical_and.reduce(arr[:, :-1] < arr[:, 1:], axis=1)

array([ True, False,  True, False,  True])

请注意，logical_and.reduce等效于all方法。

累积与reduce有关，例如与累积有关。它产生具有中间“累加”值的相同大小的数组：

In [102]:
arr = np.arange(15).reshape((3, 5))

In [103]:
np.add.accumulate(arr, axis=1)

array([[ 0,  1,  3,  6, 10],
       [ 5, 11, 18, 26, 35],
       [10, 21, 33, 46, 60]])

outer执行两个数组之间的成对叉积：

In [104]:
arr = np.arange(3).repeat([1, 2, 2])

In [105]:
arr

array([0, 1, 1, 2, 2])

In [106]:
np.multiply.outer(arr, np.arange(5))

array([[0, 0, 0, 0, 0],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 2, 4, 6, 8],
       [0, 2, 4, 6, 8]])

外部的输出将具有一个维度，该维度是输入的维度之和：

In [107]:
x, y = np.random.randn(3, 4), np.random.randn(5)

In [108]:
result = np.subtract.outer(x, y)

In [109]:
result.shape

(3, 4, 5)

最后一种方法reduceat执行“局部归约”，本质上是对数组进行分组操作，其中将数组的切片聚合在一起。它接受一系列“ bin边”，这些边指示如何拆分和汇总值：

In [110]:
arr = np.arange(10)

In [111]:
np.add.reduceat(arr, [0, 5, 8])

array([10, 18, 17])

结果是对arr [0：5]，arr [5：8]和arr [8：]执行的减少量（此处为总和）。与其他方法一样，您可以传递轴参数：

In [112]:
arr = np.multiply.outer(np.arange(4), np.arange(5))

In [113]:
arr

array([[ 0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4],
       [ 0,  2,  4,  6,  8],
       [ 0,  3,  6,  9, 12]])

In [114]:
np.add.reduceat(arr, [0, 2, 4], axis=1)

array([[ 0,  0,  0],
       [ 1,  5,  4],
       [ 2, 10,  8],
       [ 3, 15, 12]])

有关ufunc方法的部分列表，请参见表A-2。

### 用Python编写新的ufunc

有许多用于创建自己的NumPy ufuncs的工具。最通用的方法是使用NumPy C API，但这超出了本书的范围。在本节中，我们将介绍纯Python ufunc。

numpy.frompyfunc接受Python函数以及输入和输出数量的规范。例如，将一个简单的函数逐个元素地添加为：

In [115]:
def add_elements(x, y):
    return x + y

In [116]:
add_them = np.frompyfunc(add_elements, 2, 1)

In [117]:
add_them(np.arange(8), np.arange(8))

array([0, 2, 4, 6, 8, 10, 12, 14], dtype=object)

使用frompyfunc创建的函数总是返回Python对象数组，这可能很不方便。幸运的是，还有一个替代的（但功能稍差一些）函数numpy.vectorize，它允许您指定输出类型：

In [118]:
add_them = np.vectorize(add_elements, otypes=[np.float64])

In [119]:
add_them(np.arange(8), np.arange(8))

array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14.])

这些函数提供了一种创建类似ufunc的函数的方法，但是它们非常慢，因为它们需要Python函数调用来计算每个元素，这比NumPy基于C的ufunc循环要慢得多：

In [120]:
arr = np.random.randn(10000)

In [121]:
%timeit add_them(arr, arr)

2.09 ms ± 543 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [122]:
%timeit np.add(arr, arr)

6.66 µs ± 911 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


在本章的后面，我们将展示如何使用Numba项目在Python中创建快速ufunc。

### A.5 结构化和记录阵列

到目前为止，您可能已经注意到ndarray是同构数据容器。也就是说，它代表一个内存块，其中每个元素占用相同数量的字节（由dtype确定）。从表面上看，这似乎不允许您表示异构或类似表格的数据。结构化数组是一个ndarray，其中每个元素都可以视为代表C中的结构（因此称为“结构化”名称）或SQL表中具有多个命名字段的行：

In [123]:
dtype = [('x', np.float64), ('y', np.int32)]

In [124]:
sarr = np.array([(1.5, 6), (np.pi, -2)], dtype=(dtype))

In [125]:
sarr

array([(1.5       ,  6), (3.14159265, -2)],
      dtype=[('x', '<f8'), ('y', '<i4')])

有几种指定结构化dtype的方法（请参见在线NumPy文档）。一种典型的方法是使用（field_name，field_data_type）的元组列表。现在，数组的元素是类似元组的对象，其元素可以像字典一样进行访问：

In [126]:
sarr[0]

(1.5, 6)

In [127]:
sarr[0]['y']

6

字段名称存储在dtype.names属性中。当您访问结构化数组上的字段时，将返回数据的跨步视图，因此不会复制任何内容：

In [128]:
sarr['x']

array([1.5       , 3.14159265])

### 嵌套dtype和多维字段

当指定结构化dtype时，您还可以传递形状（作为int或tuple）：

In [129]:
dtype = [('x', np.int64, 3), ('y', np.int32)]

In [130]:
arr = np.zeros(4, dtype=dtype)

In [131]:
arr

array([([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0)],
      dtype=[('x', '<i8', (3,)), ('y', '<i4')])

在这种情况下，x字段现在指向每个记录的长度为3的数组：

In [132]:
arr[0]['x']

array([0, 0, 0])

方便地，访问arr ['x']然后返回一个二维数组，而不是前面的示例中的一维数组：

In [133]:
arr['x']

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

这使您可以将更复杂的嵌套结构表示为数组中的单个内存块。您还可以嵌套dtype以使结构更复杂。这是一个例子：

In [134]:
dtype = [('x', [('a', 'f8'), ('b', 'f4')]), ('y', np.int32)]

In [135]:
data = np.array([((1, 2), 5), ((3, 4), 6)], dtype=dtype)

In [136]:
data['x']

array([(1., 2.), (3., 4.)], dtype=[('a', '<f8'), ('b', '<f4')])

In [137]:
data['y']

array([5, 6], dtype=int32)

In [138]:
data['x']['a']

array([1., 3.])

pandas DataFrame不直接支持此功能，尽管它类似于分层索引。

### 为什么要使用结构化数组？

与pandas DataFrame相比，NumPy结构化数组是一个相对较低级别的工具。它们提供了一种将内存块解释为具有任意复杂嵌套列的表格结构的方法。由于数组中的每个元素在内存中均以固定的字节数表示，因此结构化数组提供了一种非常快速有效的方式来往磁盘（包括内存映射）写入数据，在网络上传输数据以及其他类似用途。

作为结构化数组的另一种常用用法，将数据文件作为固定长度的记录字节流写入是用C和C ++代码序列化数据的一种常用方法，这在工业遗留系统中很常见。只要知道文件的格式（每个记录的大小以及每个元素的顺序，字节大小和数据类型），就可以使用np.fromfile将数据读入内存。像这样的专门用途不在本书的讨论范围之内，但是值得一提的是这样的事情是可能的。

## A.6 有关排序的更多信息

像Python的内置列表一样，ndarray排序实例方法是就地排序，这意味着无需重新生成数组即可重新排列数组内容：

In [139]:
arr = np.random.randn(6)
arr.sort()
arr

array([-1.08199644,  0.37588273,  0.80139193,  1.13969136,  1.28881614,
        1.84126094])

就地对数组进行排序时，请记住，如果该数组是另一个ndarray上的视图，则将修改原始数组：

In [140]:
arr = np.random.randn(3, 5)

In [141]:
arr

array([[-0.33176812, -1.47108206,  0.87050269, -0.08468875, -1.13286962],
       [-1.01114869, -0.34357617,  2.17140268,  0.12337075, -0.01893118],
       [ 0.17731791,  0.7423957 ,  0.85475634,  1.03797268, -0.32899594]])

In [142]:
arr[:, 0].sort()  # Sort first column values in-place

In [143]:
arr

array([[-1.01114869, -1.47108206,  0.87050269, -0.08468875, -1.13286962],
       [-0.33176812, -0.34357617,  2.17140268,  0.12337075, -0.01893118],
       [ 0.17731791,  0.7423957 ,  0.85475634,  1.03797268, -0.32899594]])

另一方面，numpy.sort创建一个新的，已排序的数组副本。否则，它接受与ndarray.sort相同的参数（例如kind）：

In [144]:
arr = np.random.randn(5)

In [145]:
arr

array([-1.11807759, -0.24152521, -2.0051193 ,  0.73788753, -1.06137462])

In [146]:
np.sort(arr)

array([-2.0051193 , -1.11807759, -1.06137462, -0.24152521,  0.73788753])

In [147]:
arr

array([-1.11807759, -0.24152521, -2.0051193 ,  0.73788753, -1.06137462])

In [148]:
np.sort(arr)

array([-2.0051193 , -1.11807759, -1.06137462, -0.24152521,  0.73788753])

In [149]:
arr

array([-1.11807759, -0.24152521, -2.0051193 ,  0.73788753, -1.06137462])

所有这些排序方法都带有一个axis参数，用于沿传递的轴独立地对数据段进行排序：

In [150]:
arr = np.random.randn(3, 5)

In [151]:
arr

array([[ 0.59545348, -0.26822958,  1.33885804, -0.18715572,  0.91108374],
       [-0.32150045,  1.00543901, -0.51683937,  1.19251887, -0.19893404],
       [ 0.39691349, -1.76381537,  0.60709023, -0.22215536, -0.21707838]])

In [152]:
arr.sort(axis=1)

In [153]:
arr

array([[-0.26822958, -0.18715572,  0.59545348,  0.91108374,  1.33885804],
       [-0.51683937, -0.32150045, -0.19893404,  1.00543901,  1.19251887],
       [-1.76381537, -0.22215536, -0.21707838,  0.39691349,  0.60709023]])

您可能会注意到，排序方法都没有一个选项可以按降序排序。实际上，这是一个问题，因为数组切片会产生视图，因此不会产生副本或需要任何计算工作。 许多Python用户熟悉“技巧”，即对于列表值，values [::-1]以相反的顺序返回列表。 ndarrays也是如此：

In [154]:
arr[:, ::-1]

array([[ 1.33885804,  0.91108374,  0.59545348, -0.18715572, -0.26822958],
       [ 1.19251887,  1.00543901, -0.19893404, -0.32150045, -0.51683937],
       [ 0.60709023,  0.39691349, -0.21707838, -0.22215536, -1.76381537]])

### 间接排序：argsort和lexsort

在数据分析中，您可能需要通过一个或多个键对数据集进行重新排序。例如，关于某些学生的数据表可能需要按姓氏，然后按名字排序。这是一个间接排序的示例，如果您已经阅读了与熊猫相关的章节，那么您已经看到了许多更高级别的示例。给定一个或多个键（一个值数组或多个值数组），您希望获得一个整数索引数组（我通常将其称为索引器），告诉您如何将数据重新排序为排序后的顺序。两种方法是argsort和numpy.lexsort。举个例子：

In [155]:
values = np.array([5, 0, 1, 3, 2])

In [156]:
indexer = values.argsort()

In [157]:
indexer

array([1, 2, 4, 3, 0])

In [158]:
values[indexer]

array([0, 1, 2, 3, 5])

作为一个更复杂的示例，此代码按其第一行对二维数组进行重新排序：

In [159]:
arr = np.random.randn(3, 5)

arr[0] = values

In [160]:
arr

array([[ 5.        ,  0.        ,  1.        ,  3.        ,  2.        ],
       [-0.36360302, -0.13775933,  2.17773731, -0.47280687,  0.8356152 ],
       [-0.20885016,  0.23159352,  0.72798172, -1.3918432 ,  1.99558262]])

In [161]:
arr[:, arr[0].argsort()]

array([[ 0.        ,  1.        ,  2.        ,  3.        ,  5.        ],
       [-0.13775933,  2.17773731,  0.8356152 , -0.47280687, -0.36360302],
       [ 0.23159352,  0.72798172,  1.99558262, -1.3918432 , -0.20885016]])

lexsort与argsort相似，但是它对多个键数组执行间接字典排序。假设我们要对一些由名字和姓氏标识的数据进行排序：

In [162]:
first_name = np.array(['Bob', 'Jane', 'Steve', 'Bill', 'Barbara'])

In [163]:
last_name = np.array(['Jones', 'Arnold', 'Arnold', 'Jones', 'Walters'])

In [164]:
sorter = np.lexsort([first_name, last_name])

In [165]:
sorter

array([1, 2, 3, 0, 4])

In [166]:
list(zip(last_name[sorter], first_name[sorter]))

[('Arnold', 'Jane'),
 ('Arnold', 'Steve'),
 ('Jones', 'Bill'),
 ('Jones', 'Bob'),
 ('Walters', 'Barbara')]

lexsort第一次使用时可能会有些混乱，因为使用键对数据进行排序的顺序是从最后一个传递的数组开始的。在此，last_name在first_name之前使用。

> Series和DataFrame的sort_values方法之类的pandas方法是通过这些函数的变体实现的（还必须考虑缺失值）。

### 替代排序算法

稳定的排序算法可保留相等元素的相对位置。这在相对排序有意义的间接排序中尤其重要：

In [167]:
values = np.array(['2:first', '2:second', '1:first', '1:second', '1:third'])

In [168]:
key = np.array([2, 2, 1, 1, 1])

In [169]:
indexer = key.argsort(kind='mergesort')

In [170]:
indexer

array([2, 3, 4, 0, 1])

In [171]:
values.take(indexer)

array(['1:first', '1:second', '1:third', '2:first', '2:second'],
      dtype='<U8')

唯一可用的稳定排序是mergesort，它可以保证O（n log n）性能（针对复杂性增强），但其性能平均比默认的quicksort方法差。有关可用方法及其相对性能（和性能保证）的摘要，请参见表A-3。大多数用户都不必考虑这一点，但是知道它在那里很有用。

### 部分排序数组

排序的目标之一可以是确定数组中的最大或最小元素。NumPy具有优化的方法numpy.partition和np.argpartition，用于围绕第k个最小元素对数组进行分区：

In [172]:
np.random.seed(12345)

In [173]:
arr = np.random.randn(20)

In [174]:
arr

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057,
        1.39340583,  0.09290788,  0.28174615,  0.76902257,  1.24643474,
        1.00718936, -1.29622111,  0.27499163,  0.22891288,  1.35291684,
        0.88642934, -2.00163731, -0.37184254,  1.66902531, -0.43856974])

In [175]:
np.partition(arr, 3)

array([-2.00163731, -1.29622111, -0.5557303 , -0.51943872, -0.37184254,
       -0.43856974, -0.20470766,  0.28174615,  0.76902257,  0.47894334,
        1.00718936,  0.09290788,  0.27499163,  0.22891288,  1.35291684,
        0.88642934,  1.39340583,  1.96578057,  1.66902531,  1.24643474])

调用partition（arr，3）之后，结果中的前三个元素是没有特定顺序的最小三个值。numpy.argpartition与numpy.arg sort类似，返回将数据重新排列为等效顺序的索引：

In [176]:
indices = np.argpartition(arr, 3)

In [177]:
indices

array([16, 11,  3,  2, 17, 19,  0,  7,  8,  1, 10,  6, 12, 13, 14, 15,  5,
        4, 18,  9])

In [178]:
arr.take(indices)

array([-2.00163731, -1.29622111, -0.5557303 , -0.51943872, -0.37184254,
       -0.43856974, -0.20470766,  0.28174615,  0.76902257,  0.47894334,
        1.00718936,  0.09290788,  0.27499163,  0.22891288,  1.35291684,
        0.88642934,  1.39340583,  1.96578057,  1.66902531,  1.24643474])

### numpy.searchsorted：在排序数组中查找元素

searchsorted是一种数组方法，它对已排序的数组执行二进制搜索，并返回该数组中需要插入值以保持排序的位置：

In [179]:
arr = np.array([0, 1, 7, 12, 15])

In [180]:
arr.searchsorted(9)

3

您还可以传递值数组以返回索引数组：

In [181]:
arr.searchsorted([0, 8, 11, 16])

array([0, 3, 3, 5])

您可能已经注意到searchsorted为0元素返回了0。这是因为默认行为是在一组相等值的左侧返回索引：

In [182]:
arr = np.array([0, 0, 0, 1, 1, 1, 1])

In [183]:
arr.searchsorted([0, 1])

array([0, 3])

In [184]:
arr.searchsorted([0, 1], side='right')

array([3, 7])

作为searchsorted的另一个应用程序，假设我们有一个0到10,000之间的值数组，以及一个我们想用来对数据进行装箱的单独的“ bucket edge”数组：

In [185]:
data = np.floor(np.random.uniform(0, 10000, size=50))

In [187]:
bins = np.array([0, 100, 1000, 5000, 10000])

In [188]:
data

array([9940., 6768., 7908., 1709.,  268., 8003., 9037.,  246., 4917.,
       5262., 5963.,  519., 8950., 7282., 8183., 5002., 8101.,  959.,
       2189., 2587., 4681., 4593., 7095., 1780., 5314., 1677., 7688.,
       9281., 6094., 1501., 4896., 3773., 8486., 9110., 3838., 3154.,
       5683., 1878., 1258., 6875., 7996., 5735., 9732., 6340., 8884.,
       4954., 3516., 7142., 5039., 2256.])

为了获得每个数据点属于哪个间隔的标签（其中1表示存储桶[0，100]），我们可以简单地使用searchsorted：

In [189]:
labels = bins.searchsorted(data)

In [190]:
labels

array([4, 4, 4, 3, 2, 4, 4, 2, 3, 4, 4, 2, 4, 4, 4, 4, 4, 2, 3, 3, 3, 3,
       4, 3, 4, 3, 4, 4, 4, 3, 3, 3, 4, 4, 3, 3, 4, 3, 3, 4, 4, 4, 4, 4,
       4, 3, 3, 4, 4, 3])

结合熊猫的groupby，可用于对数据进行分箱：

In [191]:
import pandas as pd

In [192]:
pd.Series(data).groupby(labels).mean()

2     498.000000
3    3064.277778
4    7389.035714
dtype: float64